In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(5)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

import os
import shutil
directory_out = './file/4.vehicle'
if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)
import re

# = = =  = = =  = = =

import pandas as pd

import json

# = = =  = = =  = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = =  = = =  = = =

directory_in = './file/3.part'
list_file = [file for file in sorted(list(os.walk(directory_in))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_in, file))]
for file in list_file:

    # = = =  = = =  = = =

    print('Data Loading...')
    list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(os.path.join(directory_in, file),
                                                                           header=0,
                                                                           dtype=str).fillna('')
    print('Loading Done !')
    print()

    df_input = df_input[df_input['Json_Compatibility'] != json.dumps({})].reset_index(drop=True)
    df_input['No'] = df_input['No'].astype(int)

    crawler_remain = len(df_input)

    print(f'总数量：{crawler_remain}')
    print()

    # = = =  = = =  = = =

    def crawler(crawler_tuple):
        global list_dict_correct, list_series_error, crawler_remain

        crawler_index, crawler_series = crawler_tuple

        # = = =  = = =  = = =

        try:
            list_dict_correct_temp, dict_ = [], json.loads(crawler_series['Json_Compatibility'])
            data = {'scopedContext': {'catalogDetails': {'categoryId': dict_['category_id'],
                                                         'itemId': dict_['item_id'],
                                                         'marketplaceId': dict_['market_place_id'],
                                                         'sellerName': dict_['seller_name']}}}

            for page in range(dict_['page']):

                crawler_retry = 0
                while True:
                    crawler_retry += 1

                    try:
                        url_check = crawler_series['Url']

                        resp = requests.post(f'''{dict_['api']}?module_groups=PART_FINDER&referrer=VIEWITEM&offset={page * 20}&module=COMPATIBILITY_TABLE''',
                                             data=json.dumps(data),
                                             headers=get_header(**{'Content-Type': 'application/json; charset=utf-8'}),
                                             proxies=get_proxy(),
                                             timeout=(5, 15))

                        if resp.status_code == 200:
                            list_header, list_row = [header['textSpans'][0]['text'].strip() for header in resp.json()['modules']['COMPATIBILITY_TABLE']['paginatedTable']['header']['cells']], [[dict_row['textualDisplays'][0]['textSpans'][0]['text'].strip() if 'textualDisplays' in dict_row else dict_row['textSpans'][0]['text'].strip() for dict_row in list_dict_row['cells']] for list_dict_row in resp.json()['modules']['COMPATIBILITY_TABLE']['paginatedTable']['rows']]

                            if list_header and list_row:
                                break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = =  = = =  = = =

                list_dict_correct_temp_temp = [{'Json_Compatibility': crawler_series['Json_Compatibility'],
                                                'Page': page + 1,
                                                'Row': i + 1} for i in range(len(list_row))]

                for i, row in enumerate(list_row):
                    for j, header in enumerate(list_header):
                        list_dict_correct_temp_temp[i][header] = row[j]

                list_dict_correct_temp.extend(list_dict_correct_temp_temp)

                # = = =  = = =  = = =

                if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and crawler_remain == 1 or crawler_remain % 30 == 0:
                    clear_output()
                print(f'''[第{page + 1}页] - [请求{crawler_retry}次] - [剩余{crawler_remain - 1}条] - [{file.removesuffix('.xlsx')}（{(list_file.index(file) + 1) / len(list_file) * 100:.2f}%）] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['No']}. {url_check}\n''')

                # = = =  = = =  = = =

            if not list_dict_correct_temp:
                raise

            # = = =  = = =  = = =

            with semaphore_correct:
                list_dict_correct.extend(list_dict_correct_temp)
        except:
            crawler_series['Check_Url'] = url_check

            with semaphore_error:
                list_series_error.append(crawler_series)

        # = = =  = = =  = = =

        with semaphore_remain:
            crawler_remain -= 1

    # = = =  = = =  = = =

    pool.map(crawler, df_input.iterrows())

    print('Data Outputting...')
    if list_dict_correct:
        df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Json_Compatibility', 'Page', 'Row'],
                                                                 ascending=[True, True, True],
                                                                 ignore_index=True)
        df_correct.to_excel(os.path.join(directory_out, f'''{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx'''), index=False)
    print('Output Finished !')
    print()
    if list_series_error:
        df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                               ascending=[True],
                                                               ignore_index=True)
        df_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-vehicle_error.xlsx''', index=False)
        print('- Error Occurred -')
        print()

# = = =  = = =  = = =

list_file = [file for file in sorted(list(os.walk(directory_out))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_out, file))]
for file in list_file:
    os.rename(os.path.join(directory_out, file), os.path.join(directory_out, re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '-vehicle.xlsx', file)))

# = = =  = = =  = = =

print('Done ~')

[第2页] - [请求2次] - [剩余29条] - [crawler_2（100.00%）] - [2024/12/05 23:01:25] - 4564. https://www.ebay.de/itm/304954933533?_ul=DE&_stpos=10115&orig_cvip=true

[第14页] - [请求1次] - [剩余28条] - [crawler_2（100.00%）] - [2024/12/05 23:01:26] - 4464. https://www.ebay.de/itm/305367980121?_ul=DE&_stpos=10115&orig_cvip=true

[第1页] - [请求1次] - [剩余28条] - [crawler_2（100.00%）] - [2024/12/05 23:01:26] - 4581. https://www.ebay.de/itm/305568154878?_ul=DE&_stpos=10115&orig_cvip=true

[第16页] - [请求3次] - [剩余28条] - [crawler_2（100.00%）] - [2024/12/05 23:01:26] - 4496. https://www.ebay.de/itm/305949967126?_ul=DE&_stpos=10115&orig_cvip=true

[第1页] - [请求1次] - [剩余28条] - [crawler_2（100.00%）] - [2024/12/05 23:01:26] - 4582. https://www.ebay.de/itm/375527389507?_ul=DE&_stpos=10115&orig_cvip=true

[第1页] - [请求1次] - [剩余28条] - [crawler_2（100.00%）] - [2024/12/05 23:01:26] - 4576. https://www.ebay.de/itm/394794008132?_ul=DE&_stpos=10115&orig_cvip=true

[第18页] - [请求1次] - [剩余27条] - [crawler_2（100.00%）] - [2024/12/05 23:01:26] - 4456.